In [ ]:
#import libraries
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

In [ ]:
# bring in csv files
food_security_hh = "data/data-EFK41.csv"
obesity_df = "data/data-xeuft.csv"
grand_food_security_df = "data/XVKxd.csv"



In [ ]:
# Separate out the state and county columns

In [ ]:
# Rename Columns

In [ ]:
# Get rid of what is not related to GA

In [ ]:
# Merge by Counties

In [ ]:
# Sahmirah's Visual Block

In [ ]:
# Sahmirah's Visual Block 2

In [ ]:
# Sahmirah's Visual Block 3

In [ ]:
# Kendall's Visual Block

In [ ]:
# Kendall's Visual Block 1

In [ ]:
# Kendall's Visual Block 2

In [ ]:
# Bobby's Visual Block

In [ ]:
# Bobby's Visual Block 1 

In [ ]:
# Bobby's Visual Block 2